In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['TabNet MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["TabNet(LR=0.02)"])
baseline_data = np.array(baseLine_data)
tabnet = median(baseline_data[:,0])

# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.475019,0.460187,3.081982,9.203735
1,0.205479,0.326779,2.026699,6.535589
2,0.191203,0.338217,1.955339,6.764337
3,0.148393,0.317127,1.722406,6.342540
4,0.257849,0.411071,2.270328,8.221419
5,0.279851,0.433770,2.365711,8.675393
6,0.171030,0.354524,1.849134,7.090472
7,0.501167,0.551058,3.165854,11.021160
8,0.878199,0.579147,4.190826,11.582947
9,0.981278,0.585285,4.429952,11.705696


MEDIAN:
MSE                   0.268850
MAE                   0.422420
Euclidean Distance    2.318020
Manhattan Distance    8.448406
dtype: float64
0.4224202967224643


# ABC_GAN Analysis

## ABC Pre-generator - TabNet 

In [5]:
book = sb.read_notebooks("./ABC_GAN_TabNet")
paramVal = [[1,1],[1,0.1],[1,0.01],[1,0],[0.1,1],[0.1,0.1],[0.1,0.01],[0.1,0],[0.01,1],[0.01,0.1],[0.01,0.01],[0.01,0]]
abc_mae = [[] for i in range(12)]
abc_mae_skip = [[] for i in range(12)]
abc_mae_mean = [[] for i in range(12)]
abc_mae_skip_mean = [[] for i in range(12)]
abc_weights = [[] for i in range(12)]
prior_model = [[] for i in range(12)]
abc_pre_generator = [[] for i in range(12)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[1]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[0]['value'])
    # #Divide data according to parameters 
    for i in range(12):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:
data = [[] for i in range(12)]
for i in range(12):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.26224,1.27267,0.12315,0.32825,0.32203
1,1,1,0.25144,1.07432,0.14760,0.31595,0.26913
2,1,1,0.41890,1.24208,0.12715,0.40338,0.39426
3,1,1,0.51930,1.16254,0.45957,0.63657,0.58900
4,1,1,0.44460,1.28810,0.12771,0.37620,0.45012
5,1,1,0.32573,1.19235,0.28127,0.41018,0.50470
6,1,1,0.22804,1.20343,0.12735,0.30351,0.40672
7,1,1,0.23102,1.45562,0.14964,0.38760,0.44308
8,1,1,0.53576,1.28282,0.94479,0.41346,0.44656


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.325734
ABC pre-generator MAE            1.242078
Skip Node weight                 0.147605
ABC GAN MAE                      0.387603
ABC_GAN MAE (skip connection)    0.443077
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.1,0.35488,0.78704,0.17125,0.35666,0.30502
1,1,0.1,0.32466,0.87304,0.91601,0.48066,0.30332
2,1,0.1,0.34348,0.80941,0.20607,0.46908,0.39461
3,1,0.1,0.25704,0.94214,0.81797,0.23783,0.22467
4,1,0.1,0.46143,0.77057,0.87207,0.67852,0.62076
5,1,0.1,0.28672,0.80604,0.75429,0.29414,0.24885
6,1,0.1,0.35837,0.61847,0.74792,0.46754,0.47843
7,1,0.1,0.36707,0.95131,0.15140,0.36826,0.35278
8,1,0.1,0.30916,0.66621,0.77060,0.30507,0.28372


Variance                         1.000000
Bias                             0.100000
Prior Model MAE                  0.343482
ABC pre-generator MAE            0.806040
Skip Node weight                 0.754294
ABC GAN MAE                      0.368264
ABC_GAN MAE (skip connection)    0.305021
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.01,0.36756,0.81512,0.49201,0.44731,0.29406
1,1,0.01,0.36758,0.89253,0.14603,0.31764,0.38864
2,1,0.01,0.33931,0.80807,0.51593,0.30160,0.25019
3,1,0.01,0.36606,0.91895,0.47661,0.35218,0.32829
4,1,0.01,0.33845,0.83928,0.53094,0.32283,0.33518
5,1,0.01,0.49218,0.81003,0.22425,0.50500,0.36572
6,1,0.01,0.41328,0.84767,0.21794,0.40953,0.44265
7,1,0.01,0.37879,0.75985,0.99369,0.33536,0.38595
8,1,0.01,0.43971,0.99345,0.96467,0.46655,0.52038


...

Variance                         0.010000
Bias                             0.100000
Prior Model MAE                  0.445348
ABC pre-generator MAE            0.452693
Skip Node weight                 0.408681
ABC GAN MAE                      0.440095
ABC_GAN MAE (skip connection)    0.454146
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.40036,0.40386,0.07461,0.35641,0.39069
1,0.01,0.01,0.35101,0.35347,0.00000,0.45542,0.35285
2,0.01,0.01,0.42777,0.42462,0.07418,0.44800,34.02934
3,0.01,0.01,0.37902,0.37354,0.00000,0.35164,0.37601
4,0.01,0.01,0.61890,0.62194,0.00000,0.59456,0.62372
5,0.01,0.01,0.26605,0.26287,0.42099,0.33500,0.31544
6,0.01,0.01,0.52063,0.52033,0.28321,0.55084,0.56689
7,0.01,0.01,0.31698,0.30942,0.00000,0.35166,0.31309
8,0.01,0.01,0.49732,0.49221,0.04085,0.55354,0.52319


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.400363
ABC pre-generator MAE            0.403856
Skip Node weight                 0.040846
ABC GAN MAE                      0.448003
ABC_GAN MAE (skip connection)    0.390693
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0,0.40635,0.40738,0.75797,0.43538,0.42300
1,0.01,0,0.34282,0.34160,0.00000,0.44832,0.34297
2,0.01,0,0.33980,0.33718,0.74654,0.32080,0.32708
3,0.01,0,0.29645,0.29604,0.00000,0.44265,0.29663
4,0.01,0,0.28270,0.28167,0.00000,0.22045,0.28288
5,0.01,0,0.43707,0.43997,0.14055,0.42578,0.40822
6,0.01,0,0.42221,0.42350,0.00000,0.37111,0.42248
7,0.01,0,0.32543,0.32810,0.54242,0.32254,0.32615
8,0.01,0,0.30592,0.30860,0.04881,0.24309,7.33745


Variance                         0.010000
Bias                             0.000000
Prior Model MAE                  0.339801
ABC pre-generator MAE            0.337181
Skip Node weight                 0.048808
ABC GAN MAE                      0.371106
ABC_GAN MAE (skip connection)    0.342971
dtype: float64
-------------------------------------------------------------


In [7]:
# Display TabNet Summary Tables 
data = np.array(data)
tabnetData = []
for i in range(12): 
    data[i] = np.array(data[i])
    tabnetData.append([paramVal[i][0], paramVal[i][1],tabnet,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(tabnetData, columns = ['Variance','Bias','TabNet','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,TabNet,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.41428,1.24208,0.38760,0.44308,0.14760
1,1.00,0.10,0.41428,0.80604,0.36826,0.30502,0.75429
2,1.00,0.01,0.41428,0.83928,0.35218,0.36572,0.49201
3,1.00,0.00,0.41428,0.82939,0.35123,0.36422,0.19978
4,0.10,1.00,0.41428,1.04031,0.45532,0.44616,0.48783
5,0.10,0.10,0.41428,0.40299,0.44167,0.40793,0.23177
6,0.10,0.01,0.41428,0.35918,0.39433,0.45073,0.19165
7,0.10,0.00,0.41428,0.37565,0.45641,0.47964,0.07218
8,0.01,1.00,0.41428,1.03853,0.50938,0.46778,0.19117
9,0.01,0.10,0.41428,0.45269,0.44009,0.45415,0.40868
